# 🌀 Mandelbrot Set Explorer on Tenstorrent Hardware

Interactive fractal visualization using TTNN for parallel computation.

## 🚀 Getting Started

**First time using this notebook?**
1. **Run cell 2** (dependency check) - it will automatically:
   - Check for numpy, matplotlib, and ttnn
   - Auto-install missing packages when possible
   - Provide clear instructions if manual setup needed
2. **Follow any instructions** that appear
3. **Continue to cell 4** when you see "✅ Ready to run the notebook!"

## 📦 Requirements

- **ttnn** (from TT-Metal) - Required for hardware acceleration
- **numpy** - Array operations (auto-installed if missing)
- **matplotlib** - Visualization (auto-installed if missing)

**Note:** If ttnn is not installed, cell 2 will detect this and either:
- ✅ Auto-install it from ~/tt-metal (if found), or
- 📋 Show you how to install it manually

## ✨ Features

- High-resolution Mandelbrot rendering
- Julia set variations
- Performance benchmarking
- Multiple color schemes
- Inline visualization (no external windows needed!)

## 🎮 Interactive Controls

In cells with interactive plots:
- Click to zoom into region
- R: Reset view
- C: Cycle color maps
- U: Undo zoom
- Q: Quit

## 📦 Dependency Check

Run this cell first to ensure all required packages are installed.

In [ ]:
# Check and install required dependencies
import sys
import subprocess
import os
from pathlib import Path

def check_and_install_packages():
    """Check if required packages are installed, install if missing."""
    
    # Standard packages that can be pip installed
    standard_packages = {
        'numpy': 'numpy',
        'matplotlib': 'matplotlib'
    }
    
    missing_packages = []
    
    print("🔍 Checking dependencies...\n")
    
    # Check standard packages
    for module_name, package_name in standard_packages.items():
        try:
            __import__(module_name)
            print(f"✅ {package_name} is installed")
        except ImportError:
            print(f"❌ {package_name} is not installed")
            missing_packages.append(package_name)
    
    # Check for ttnn (special case - needs tt-metal)
    ttnn_available = False
    try:
        import ttnn
        print(f"✅ ttnn is installed")
        ttnn_available = True
    except ImportError:
        print(f"❌ ttnn is not installed")
    
    # Install missing standard packages
    if missing_packages:
        print(f"\n📦 Installing missing packages: {', '.join(missing_packages)}")
        
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
            print("✅ Installed packages from requirements.txt")
        except subprocess.CalledProcessError:
            print("⚠️ Could not install from requirements.txt, installing packages directly...")
            for package in missing_packages:
                subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print("✅ Packages installed successfully")
    
    # Handle ttnn installation
    if not ttnn_available:
        print("\n" + "="*60)
        print("⚠️  TTNN NOT FOUND - Installation Required")
        print("="*60)
        
        # Check for tt-metal directory
        tt_metal_path = Path.home() / "tt-metal"
        
        if tt_metal_path.exists():
            print(f"\n✅ Found tt-metal at: {tt_metal_path}")
            print("\n📦 Attempting to install ttnn from tt-metal...")
            
            try:
                # Try to install ttnn from tt-metal
                result = subprocess.run(
                    [sys.executable, "-m", "pip", "install", "-e", str(tt_metal_path)],
                    capture_output=True,
                    text=True,
                    timeout=300  # 5 minute timeout
                )
                
                if result.returncode == 0:
                    print("✅ Successfully installed ttnn!")
                    print("\n💡 Please RESTART the notebook kernel (Kernel → Restart Kernel)")
                    print("   Then re-run this cell to verify installation.")
                    return False
                else:
                    print(f"❌ Installation failed: {result.stderr}")
                    print("\n🔧 Manual installation required:")
                    print(f"   1. Open terminal in VSCode")
                    print(f"   2. cd {tt_metal_path}")
                    print(f"   3. pip install -e .")
                    print(f"   4. Restart notebook kernel")
                    return False
                    
            except subprocess.TimeoutExpired:
                print("❌ Installation timed out (took longer than 5 minutes)")
                print("\n🔧 Please install manually:")
                print(f"   cd {tt_metal_path} && pip install -e .")
                return False
            except Exception as e:
                print(f"❌ Error during installation: {e}")
                print("\n🔧 Please install manually:")
                print(f"   cd {tt_metal_path} && pip install -e .")
                return False
        else:
            print(f"\n❌ tt-metal directory not found at: {tt_metal_path}")
            print("\n🔧 To install ttnn:")
            print("   1. Clone tt-metal:")
            print("      git clone https://github.com/tenstorrent/tt-metal.git ~/tt-metal")
            print("   2. Install dependencies:")
            print("      cd ~/tt-metal")
            print("      ./install_dependencies.sh")
            print("   3. Build tt-metal:")
            print("      ./build_metal.sh")
            print("   4. Install Python package:")
            print("      pip install -e .")
            print("   5. Restart this notebook kernel")
            return False
    
    if not missing_packages and ttnn_available:
        print("\n✅ All required packages are installed!")
        print("💡 Ready to run the notebook!")
        return True
    elif not missing_packages:
        print("\n⚠️  Standard packages OK, but ttnn still needs installation")
        return False
    else:
        print("\n✅ Standard packages installed!")
        if not ttnn_available:
            print("⚠️  Still need to install ttnn (see instructions above)")
        return False

# Run the check
ready = check_and_install_packages()

if not ready:
    print("\n" + "="*60)
    print("⏸️  SETUP INCOMPLETE - Please follow instructions above")
    print("="*60)

## 🚀 Initialize Hardware

After dependencies are installed, run this cell to initialize the Tenstorrent device.

In [ ]:
# Import libraries and initialize device
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import ttnn
from renderer import MandelbrotRenderer
from explorer import MandelbrotVisualizer

# Initialize device
device = ttnn.open_device(device_id=0)
renderer = MandelbrotRenderer(device)
viz = MandelbrotVisualizer(renderer)

print("✅ Device initialized and ready!")

## 🎨 Classic Mandelbrot Set

The famous fractal discovered by Benoit Mandelbrot. Each pixel represents a complex number `c`, and we iterate `z = z² + c` to see if it diverges.

In [ ]:
# Render classic view
mandelbrot = renderer.render(
    width=1024, height=1024,
    x_min=-2.5, x_max=1.0,
    y_min=-1.25, y_max=1.25,
    max_iter=256
)

# Display with matplotlib
fig, ax = plt.subplots(figsize=(12, 12))
color_data = np.log(mandelbrot + 1)
im = ax.imshow(color_data, cmap='hot', origin='lower', extent=[-2.5, 1.0, -1.25, 1.25])
ax.set_title('Classic Mandelbrot Set', fontsize=16)
ax.set_xlabel('Real axis')
ax.set_ylabel('Imaginary axis')
plt.colorbar(im, ax=ax, label='log(iterations)')
plt.tight_layout()
plt.show()

## 🔍 Zoom into Interesting Regions

The Mandelbrot set is infinitely detailed - zoom into any boundary region to see intricate patterns.

In [ ]:
# Zoom into the "seahorse valley" region
seahorse = renderer.render(
    width=1024, height=1024,
    x_min=-0.75, x_max=-0.735,
    y_min=0.095, y_max=0.11,
    max_iter=512
)

fig, ax = plt.subplots(figsize=(12, 12))
color_data = np.log(seahorse + 1)
im = ax.imshow(color_data, cmap='twilight', origin='lower', 
               extent=[-0.75, -0.735, 0.095, 0.11])
ax.set_title('Seahorse Valley (40x zoom)', fontsize=16)
ax.set_xlabel('Real axis')
ax.set_ylabel('Imaginary axis')
plt.colorbar(im, ax=ax, label='log(iterations)')
plt.tight_layout()
plt.show()

In [ ]:
# Zoom into the "elephant valley" region
elephant = renderer.render(
    width=1024, height=1024,
    x_min=0.25, x_max=0.35,
    y_min=0.0, y_max=0.1,
    max_iter=512
)

fig, ax = plt.subplots(figsize=(12, 12))
color_data = np.log(elephant + 1)
im = ax.imshow(color_data, cmap='viridis', origin='lower',
               extent=[0.25, 0.35, 0.0, 0.1])
ax.set_title('Elephant Valley', fontsize=16)
ax.set_xlabel('Real axis')
ax.set_ylabel('Imaginary axis')
plt.colorbar(im, ax=ax, label='log(iterations)')
plt.tight_layout()
plt.show()

## 🌀 Julia Sets

Julia sets are related to the Mandelbrot set. Instead of varying `c` and starting with `z=0`, we fix `c` and vary the starting `z` position.

In [ ]:
# Compare multiple Julia sets
c_values = [
    (-0.4, 0.6),      # Dragon-like
    (-0.8, 0.156),    # Spiral
    (0.285, 0.01),    # Dendrite
    (-0.7269, 0.1889) # Douady rabbit
]

fig, axes = plt.subplots(2, 2, figsize=(14, 14))
axes = axes.flatten()

for ax, (c_real, c_imag) in zip(axes, c_values):
    julia = renderer.render_julia(
        width=512, height=512,
        c_real=c_real, c_imag=c_imag,
        max_iter=256
    )
    
    color_data = np.log(julia + 1)
    im = ax.imshow(color_data, cmap='twilight', origin='lower')
    ax.set_title(f'Julia Set: c = {c_real:.3f} + {c_imag:.3f}i', fontsize=12)
    ax.axis('off')

plt.tight_layout()
plt.show()

## 🎨 Color Scheme Comparison

The same fractal can look dramatically different with different color maps.

In [ ]:
# Render a detailed region once
fractal = renderer.render(
    width=1024, height=1024,
    x_min=-0.8, x_max=-0.4,
    y_min=-0.2, y_max=0.2,
    max_iter=512
)

# Show with different color maps
colormaps = ['hot', 'viridis', 'twilight', 'gist_earth', 'nipy_spectral']
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

color_data = np.log(fractal + 1)

for ax, cmap in zip(axes, colormaps):
    im = ax.imshow(color_data, cmap=cmap, origin='lower')
    ax.set_title(f'Colormap: {cmap}', fontsize=12)
    ax.axis('off')

# Hide last subplot
axes[-1].axis('off')

plt.tight_layout()
plt.show()

## ⚡ Performance Benchmark

Test rendering performance at different resolutions.

In [ ]:
import time

resolutions = [
    (512, 512),
    (1024, 1024),
    (2048, 2048),
    (4096, 4096)
]

results = []

for width, height in resolutions:
    start = time.time()
    fractal = renderer.render(
        width=width, height=height,
        x_min=-2.5, x_max=1.0,
        y_min=-1.25, y_max=1.25,
        max_iter=256
    )
    elapsed = time.time() - start
    
    pixels = width * height
    mpixels_per_sec = (pixels / elapsed) / 1e6
    
    results.append({
        'resolution': f'{width}×{height}',
        'pixels': pixels,
        'time': elapsed,
        'throughput': mpixels_per_sec
    })
    
    print(f"{width}×{height}: {elapsed:.2f}s ({mpixels_per_sec:.2f} Mpixels/sec)")

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

resolutions_str = [r['resolution'] for r in results]
times = [r['time'] for r in results]
throughputs = [r['throughput'] for r in results]

ax1.bar(resolutions_str, times, color='steelblue')
ax1.set_ylabel('Time (seconds)')
ax1.set_title('Render Time vs Resolution')
ax1.tick_params(axis='x', rotation=45)

ax2.bar(resolutions_str, throughputs, color='coral')
ax2.set_ylabel('Throughput (Mpixels/sec)')
ax2.set_title('Throughput vs Resolution')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 🎬 Create a Zoom Animation Sequence

Render multiple frames at different zoom levels.

In [ ]:
# Define zoom sequence (center_x, center_y, zoom_factor)
zoom_sequence = [
    (-0.5, 0.0, 1),
    (-0.5, 0.0, 2),
    (-0.6, 0.0, 4),
    (-0.7, 0.0, 8),
    (-0.75, 0.1, 16),
    (-0.75, 0.1, 32)
]

frames = []

for x_center, y_center, zoom in zoom_sequence:
    x_range = 3.5 / zoom
    y_range = 2.5 / zoom
    
    x_min = x_center - x_range / 2
    x_max = x_center + x_range / 2
    y_min = y_center - y_range / 2
    y_max = y_center + y_range / 2
    
    max_iter = int(256 * (1 + np.log10(zoom)))
    
    print(f"Rendering zoom level {zoom}x (max_iter={max_iter})...")
    fractal = renderer.render(
        width=768, height=768,
        x_min=x_min, x_max=x_max,
        y_min=y_min, y_max=y_max,
        max_iter=max_iter
    )
    frames.append((fractal, zoom))

# Display all frames
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for ax, (fractal, zoom) in zip(axes, frames):
    color_data = np.log(fractal + 1)
    ax.imshow(color_data, cmap='hot', origin='lower')
    ax.set_title(f'Zoom: {zoom}x', fontsize=12)
    ax.axis('off')

plt.tight_layout()
plt.show()

print("\n✅ Zoom sequence complete!")

## 🧪 Experiment: Custom Region

Explore any region you want! Modify the parameters below.

In [ ]:
# 🎯 YOUR TURN: Modify these values to explore!
custom_fractal = renderer.render(
    width=1024,
    height=1024,
    x_min=-1.0,      # ⬅️ Change these
    x_max=-0.0,      # ⬅️ Change these  
    y_min=-0.5,      # ⬅️ Change these
    y_max=0.5,       # ⬅️ Change these
    max_iter=512
)

fig, ax = plt.subplots(figsize=(12, 12))
color_data = np.log(custom_fractal + 1)
im = ax.imshow(color_data, cmap='nipy_spectral', origin='lower')
ax.set_title('Custom Exploration', fontsize=16)
ax.set_xlabel('Real axis')
ax.set_ylabel('Imaginary axis')
plt.colorbar(im, ax=ax, label='log(iterations)')
plt.tight_layout()
plt.show()

## 🧹 Cleanup

Always close the device when done.

In [ ]:
# Close device
ttnn.close_device(device)
print("✅ Device closed. All done!")

---

## 📚 Additional Resources

**Interesting coordinates to explore:**
- Seahorse Valley: `x=-0.75 to -0.735, y=0.095 to 0.11`
- Elephant Valley: `x=0.25 to 0.35, y=0.0 to 0.1`
- Spiral: `x=-0.7, y=0.27 (zoom in)`
- Mini Mandelbrot: `x=-0.1592, y=-1.0317 (deep zoom)`

**Julia set c values to try:**
- `-0.4 + 0.6i` (dragon-like)
- `-0.8 + 0.156i` (spiral)
- `0.285 + 0.01i` (dendrite)
- `-0.7269 + 0.1889i` (Douady rabbit)
- `-0.835 - 0.2321i` (San Marco)

**Performance tips:**
- Start with lower resolution (512×512) for exploration
- Increase `max_iter` for deeper zooms (2x zoom ≈ 2x iterations)
- Use simpler colormaps (hot, viridis) for faster rendering